In [1]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def fermi_function(delta_payoff, beta):
    return 1 / (1 + np.exp(beta * delta_payoff))

def assign_strategies_random(G):
    num_nodes = len(G.nodes())
    strategies = [0] * (num_nodes // 2) + [1] * (num_nodes // 2)
    random.shuffle(strategies)

    for node, strategy in zip(G.nodes(), strategies):
        G.nodes[node]['strategy'] = strategy

def initialize_reputation(G):
    for node in G.nodes():
        G.nodes[node]['reputation'] = random.randint(0, 20)

def update_reputation(G):
    for node in G.nodes():
        if G.nodes[node]['strategy'] == 1:
            G.nodes[node]['reputation'] += 1
        elif G.nodes[node]['strategy'] == 0:
            G.nodes[node]['reputation'] -= 1

        G.nodes[node]['reputation'] = max(0, min(20, G.nodes[node]['reputation']))

def play_game(G, r):
    payoffs = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        neighbors.append(node)
        neighbor_strategies = [G.nodes[neighbor]['strategy'] for neighbor in neighbors]
        defect_count = neighbor_strategies.count(0)
        cooperation_count = neighbor_strategies.count(1)
        N = cooperation_count + defect_count

        reputations = [G.nodes[neighbor]['reputation'] for neighbor in neighbors]
        max_reputation = max(reputations)
        average_reputation = sum(reputations) / len(reputations)

        if cooperation_count == 0:
            cooperat_payoff = 0
            defect_payoff = 0
        else:
            defect_payoff = (r * cooperation_count) / N
            cooperat_payoff = defect_payoff - 1

        payoffs[node] = {'Defect': defect_payoff, 'Cooperate': cooperat_payoff}
    for node in G.nodes():
        G.nodes[node]['payoff'] = payoffs[node]['Defect'] if G.nodes[node]['strategy'] == 0 else payoffs[node]['Cooperate']

def evolutionary_game_async(G, t, r, beta, reputation_threshold, b):
    assign_strategies_random(G)
    initialize_reputation(G)

    strategy_proportions = {'Cooperate': [], 'Defect': []}

    def record_strategy_proportions(layer, strategy_proportions):
        strategy_counts = {'Cooperate': 0, 'Defect': 0}
        for node in layer.nodes():
            strategy_counts['Cooperate' if layer.nodes[node]['strategy'] == 1 else 'Defect'] += 1
        strategy_proportions['Cooperate'].append(strategy_counts['Cooperate'] / len(layer.nodes()))
        strategy_proportions['Defect'].append(strategy_counts['Defect'] / len(layer.nodes()))

    record_strategy_proportions(G, strategy_proportions)

    for node in G.nodes():
        G.nodes[node]['payoff'] = 0

    for iteration in range(t):
        for _ in range(3600):
            selected_nodes = random.sample(list(G.nodes()), 1)
            for node in selected_nodes:
                neighbors = list(G.neighbors(node))
                neighbors.append(node)
                neighbor_reputations = [G.nodes[neighbor]['reputation'] for neighbor in neighbors]
                total_reputation = sum(neighbor_reputations)

                selected_neighbor = random.choice(neighbors)
                average_reputation = sum(neighbor_reputations) / len(neighbor_reputations)

                neighbor_strategies = [G.nodes[neighbor]['strategy'] for neighbor in neighbors]
                cooperation_count = neighbor_strategies.count(1)
                defect_count = neighbor_strategies.count(0)

                if average_reputation < reputation_threshold:
                    for neighbor in neighbors:
                        if G.nodes[neighbor]['strategy'] == 0:
                            a = (20 - (G.nodes[neighbor]['reputation'] / reputation_threshold))
                            G.nodes[neighbor]['payoff'] -= b
                        my_fitness = c * G.nodes[node]['payoff'] + (1 - c) * (
                                G.nodes[node]['reputation'] - 0) / 20
                        neighbor_fitness = c * G.nodes[selected_neighbor]['payoff'] + (1 - c) * (
                                G.nodes[selected_neighbor]['reputation'] - 0) / 20
                        delta_fitness = my_fitness - neighbor_fitness
                        prob_change = fermi_function(delta_fitness, beta)
                    if random.random() < prob_change:
                        new_strategy = G.nodes[selected_neighbor]['strategy']
                        G.nodes[node]['strategy'] = new_strategy
                else:
                    my_fitness = c * G.nodes[node]['payoff'] + (1 - c) * (
                            G.nodes[node]['reputation'] - reputation_threshold) / 20
                    neighbor_fitness = c * G.nodes[selected_neighbor]['payoff'] + (1 - c) * (
                            G.nodes[selected_neighbor]['reputation'] - reputation_threshold) / 20
                    delta_fitness = my_fitness - neighbor_fitness
                    prob_change = fermi_function(delta_fitness, beta)
                    if random.random() < prob_change:
                        new_strategy = G.nodes[selected_neighbor]['strategy']
                        G.nodes[node]['strategy'] = new_strategy

        play_game(G, r)
        update_reputation(G)

        record_strategy_proportions(G, strategy_proportions)

        if strategy_proportions['Cooperate'][-1] == 1:
            break

    return strategy_proportions

# 设置网络和参数
L = 60
G = nx.grid_2d_graph(L, L, periodic=True)
beta = 2
reputation_threshold = 0
r = 2.5
b = 0
c_values = np.arange(0, 1.1, 0.1)

for c in c_values:
    strategy_proportions = evolutionary_game_async(G, 10000, r, beta, reputation_threshold, b)

    if strategy_proportions['Cooperate'][-1] < 1:
        final_ratio = np.mean(strategy_proportions['Cooperate'][-500:])
    else:
        final_ratio = 1

    print(f"c={c}, 合作比例={final_ratio}")

c=0.0, 合作比例=1
c=0.1, 合作比例=1
c=0.2, 合作比例=1
c=0.30000000000000004, 合作比例=1


: 